In [ ]:
import sys
import warnings
warnings.filterwarnings('ignore')

rateencoderspikecount = 6 #was 6
isiencoderspikecount = 6 #was 6
my_epoch=11
encoder_type = "Rate"


if __name__ == "__main__":
    if len(sys.argv) > 4:
        rateencoderspikecount = sys.argv[1]
        isiencoderspikecount = sys.argv[2]
        my_epoch = sys.argv[3]
        encoder_type = sys.argv[4]

        print("Parameters: rateencoderspikecount, isiencoderspikecount, my_epoch, encoder_type")
        print("Encoder types: Rate, TTFS, ISI, TTFS-Phase, ISI-Phase")
        print(rateencoderspikecount, isiencoderspikecount, my_epoch, encoder_type)
    else:
        print("Usage: ipython 'filename.py' <argument1> <argument2>")

rateencoderspikecount = int(rateencoderspikecount)
isiencoderspikecount = int(isiencoderspikecount)
my_epoch = int(my_epoch)

#!/usr/bin/env python
# coding: utf-8

# This notebook uses CIFAR-10 and CNN to verify the efficiency of encoding schems.

# Block of importing libraries.

# In[1]:


import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from keras.datasets import cifar10
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from keras import layers
from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import time
import scipy.io
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.datasets import mnist
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import Adam
#because of multiclass datasets
from tensorflow.keras.utils import to_categorical
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import tensorflow as tf


# Normalization function.

# In[2]:


def feature_normalize(X):

    # feature_normalize: Normalizes the features in X
    # feature_normalize(X): returns a normalized version of X where the mean value of each feature is 0 and the standard deviation is 1.
    # This is often a good preprocessing step to do when working with learning algorithms.

    mu     = 0 # mean of dataset
    sigma  = 0 #std dev
    mu     = np.mean(X)
    sigma  = np.std(X)
    X      = (X - mu) / sigma #normalization func
    X_norm = X

    return X_norm


# TTFS encoder.

# In[3]:


#Input is any any-d array (ideally 1d) that contains input currents
#Returns outputs a 1d array with each output being a different encoding window's time-to-spike.
def TTFS_Encoder(input, tau = 20, thr = 0.2, tmax = 290.2, epsilon = 1e-7):
  idx = input < thr #taking all input values below threshold...
  input = np.clip(input, thr + epsilon, 1e9) #then "squeezing" the inputs to a range between threshold and 1*10^9. Below and upper values become the min or max of the new value range
  output = tau * np.log(input / (input - thr)) # This equation shows that the output (time) for the spike to fire is caused by both the time it takes for the capacitor to fully charge (tau) and the distance/how long it takes to get to the threshold voltage given an input current (log(expression))
  output[idx] = tmax # set all inputs that are below threshold to a fixed time value: 290.2 to basically separate the inputs above thresholds from those below it
  return output


# Test the encoder with simple input.

# In[4]:


TestResult = TTFS_Encoder(np.array([[1, 2, 4, 8]]))
TestResult


# Visualize the function of TTFS encoder.

# In[5]:


#Plotting the TTFS encoder
input_x = np.linspace(1.0, 3.0, num=20) #array generation function: start, end, num. of in-betweens
plt.plot(input_x, TTFS_Encoder(np.array([input_x]))[0], 'ro')
plt.xlabel('Input', fontsize = 22)
plt.ylabel('Output', fontsize = 22)
plt.rc('xtick',labelsize=18) #rc is default setting config of graph properties
plt.rc('ytick',labelsize=18) #rc is default setting
#plt.show()


# It works well. Move to the ISI encoder. This encoder is using the parallel structure which is easier to be implemented is circuit level and is what I have implemented in circuit.

# In[6]:


#Sets output to length param. N; for each N value in the output array, switch the value to the output of tau*log(expr) of TTFS func with diff. threshold values for each.

#input is 2d array with 3 columns; each row is a separate encoding window with three different input currents
#output is a 2d array with 3 columns modified by threshold; each row is three different time-to-spikes relative to the time of when the encoding begun.
def ISI_Encoder(input, N = isiencoderspikecount):
  output = np.zeros(N)
  for j in range(N):
    output[j] = TTFS_Encoder(np.array([input]), thr = 0.2 + 0.05 * j) #Each row of the 2d array is input of TTFS func. -> 1d array as input for func.
  return output #outputs N (3 for this example) time-to-spikes for each encoding window. Output is probably a 2d array.


# Test the ISI encoder with simple input.

# In[7]:


ISITestResult = ISI_Encoder(3)
ISITestResult


# Visualize the ISI encoder.

# In[8]:


input_x = np.linspace(1.0, isiencoderspikecount, num=10)
ISI_spike_plot = np.zeros((10, isiencoderspikecount))
for i in range(10):
  ISI_spike_plot[i] = ISI_Encoder(input_x[i])
#print(ISI_spike_plot)
#Plot the ISI using a spike-like graph
plt.eventplot(ISI_spike_plot, colors=['C{}'.format(i) for i in range(ISI_spike_plot.shape[0])])
plt.xlabel('Input', fontsize = 22)
plt.ylabel('Output', fontsize = 22)
plt.rc('xtick',labelsize=18)
plt.rc('ytick',labelsize=18)
#plt.show()


# In[9]:


#Creates a reference table to save time by pre-creating spike values at corresponding input currents
ISI_SpikesList = np.zeros((256, isiencoderspikecount)) #!!!Revert back to 3
for i in range(256):
  ISI_SpikesList[i] = ISI_Encoder(i/32) #i/32 just is auto-scaler to fit image dataset settings
#print(ISI_SpikesList)


# It works well. Move to Latency-phase encoder.

# In[10]:


def TTFS_Phase_Encoder(input, SMO_freq = 5000, TMAX = 295):
  #1. Create the maximums/peaks of the sinuodal function based off of the period/interval magnitude and the maximum time.

  #print(SMO_freq)
  SMO_interval = 1000 / SMO_freq
  #print(SMO_interval)
  SMO_max = np.arange(0, TMAX - 0.1, SMO_interval)
  #print(SMO_max)

  #2. Then, process the input in a TTFS encoder

  TT = TTFS_Encoder(input)

  #3. Align the TTFS output to be at the next local maximums of the sinuoidal function
  for i in range(SMO_max.shape[0]):
    if SMO_max[i] >= TT:
      TT = SMO_max[i]
      break
  return TT # Return the newly aligned output times to first spikes


# Test the TTFS_Phase_Encoder with simple input.

# In[11]:


TTFS_PhaseTestResult = TTFS_Phase_Encoder(np.array([2]))
TTFS_PhaseTestResult


# visualize the TTFS-phase encoder.

# In[12]:


input_x = np.linspace(1.0, 6.0, num=20)
output_x = np.zeros(20)
#graph the TTFS phase aligned outputs.
for i in range(20):
  output_x[i] = TTFS_Phase_Encoder(np.array([input_x[i]]))
plt.plot(input_x, output_x, 'ro')
plt.xlabel('Input', fontsize = 22)
plt.ylabel('Output', fontsize = 22)
plt.rc('xtick',labelsize=18)
plt.rc('ytick',labelsize=18)
#plt.show()


# In[13]:


TTFS_Phase_SpikesList = np.zeros(256)
for i in range(256):
  TTFS_Phase_SpikesList[i] = TTFS_Phase_Encoder(np.array([i]))


# It works well. Move to ISI_Phase Encoder.


# In[14]:


#Same as TTFS encoder. 1. create SMO intervals based on SMO frequency. Also define the local maximums within the time range.
#2. Input a 1d or 2d array, with each row being a separate encoding window, into the ISI encoder. 3. gamma align each output to the next local max
def ISI_Phase_Encoder(input, SMO_freq = 5000, TMAX = 295):
  SMO_interval = 1000 / SMO_freq
  SMO_max = np.arange(0, TMAX - 0.1, SMO_interval)
  TT = ISI_Encoder(input) ##TT is just a 2d array (encoding windows (row) by each time-to-spike (column))
  for i in range(TT.shape[0]):
    for k in range(SMO_max.shape[0]):
      if SMO_max[k] >= TT[i]: #which SMO max is just a little bit more than the corresponding time of spike?
        TT[i] = SMO_max[k] # gamma align
        break
  return TT # return a gamma aligned


# Test the ISI_Phase Encoder with simple input.

# In[15]:


ISI_PhaseTestResult = ISI_Phase_Encoder(3)
ISI_PhaseTestResult


# Visualize the ISI-Phase Encoder.

# In[16]:


input_x = np.linspace(1.0, isiencoderspikecount, num=10)
ISI_Phase_spike_plot = np.zeros((10, isiencoderspikecount))
for i in range(10):
  ISI_Phase_spike_plot[i] = ISI_Phase_Encoder(input_x[i])
#print(ISI_Phase_spike_plot)

#Visualizing the ISI-Phase Encoder
plt.eventplot(ISI_Phase_spike_plot, colors=['C{}'.format(i) for i in range(ISI_Phase_spike_plot.shape[0])])
plt.xlabel('Input', fontsize = 22)
plt.ylabel('Output', fontsize = 22)
plt.rc('xtick',labelsize=18)
plt.rc('ytick',labelsize=18)
#plt.show()


# In[17]:


ISI_Phase_SpikesList = np.zeros((256, isiencoderspikecount)) #!!!! Revert back 6 to 3
for i in range(256): #Individually creating time-to-spikes for the lookup table for each possible input current value (i)
  ISI_Phase_SpikesList[i] = ISI_Phase_Encoder(i/32) #!!!!removed /32 after i in ISI_Phase_Encoder parantheses


# In[18]:


"""get_ipython().run_line_magic('tensorflow_version', '2.x')
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))"""


# Download and process dataset

# In[19]:


(x_train, y_train), (x_test, y_test) = cifar10.load_data()

#Start of data augmentation code

from tensorflow.keras.preprocessing.image import ImageDataGenerator


#print(x_train.shape)
labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
fig, axes = plt.subplots(ncols=7, nrows=3, figsize=(17, 8))
index = 0

#Displaying 21 different labels/nums.? on the screen
for i in range(3):
    for j in range(7):
        axes[i,j].set_title(labels[y_train[index][0]])
        axes[i,j].imshow(x_train[index]) #just displays 2d/3d arrays into images (like heatmaps)
        axes[i,j].get_xaxis().set_visible(False)
        axes[i,j].get_yaxis().set_visible(False)
        index += 1
#plt.show()

#Converting the x training and testing images into grayscale (from BGR)
#print(x_train)
#print("Max {} Min {}".format(x_train.max(), x_train.min()))
x_train = np.array([cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in x_train])
x_test = np.array([cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in x_test])
#print("Max {} Min {}".format(x_train.max(), x_train.min()))
#print(x_train)
#print(x_train.shape)

fig, axes = plt.subplots(ncols=7, nrows=3, figsize=(17, 8))
index = 0
#Displaying 21 different images on the screen
for i in range(3):
    for j in range(7):
        axes[i,j].set_title(labels[y_train[index][0]])
        axes[i,j].imshow(x_train[index], cmap='gray')
        axes[i,j].get_xaxis().set_visible(False) #Basically hides features like tick marks of x-axis and y-axis
        axes[i,j].get_yaxis().set_visible(False)
        index += 1
#plt.show()

#One hot encoder basically assigns numerical values to categorical variables
one_hot_encoder = OneHotEncoder(sparse_output=False)
one_hot_encoder.fit(y_train)
y_train = one_hot_encoder.transform(y_train)
y_test = one_hot_encoder.transform(y_test)
#print(y_train.shape)

if encoder_type == "Rate":

    # Rate Encoding.

    # In[ ]:


    #RUN ALL CODE ABOVE THIS BLOCK

    #Limited number of spikes
    #print(x_train)
    #print(x_train.max(), x_train.min())

    #256/32/8=1 -> Below code just normalizes the x_train element values to between 0 and 1

    x_train = np.rint((x_train / 255) * rateencoderspikecount).astype(int)
    x_test = np.rint((x_test / 255) * rateencoderspikecount).astype(int)
    #print(x_train)
    #print(x_train.shape)

    #Reshape dataset
    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1) #representing grayscale as a dimension of 1 at the end of x_train's shape
    x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)
    input_shape = (x_train.shape[1], x_train.shape[2], 1)
    #print(input_shape)


"""elif encoder_type == "TTFS":

    # TTFS Encoding

    # In[ ]:


    #shrink the dataset

    #Encode the dataset
    x_train = TTFS_Encoder(x_train) #creates 1d array with each element being in a different encoding window
    x_test = TTFS_Encoder(x_test)

    #Normalize the dataset
    x_train = feature_normalize(x_train)
    x_test = feature_normalize(x_test)
    #print(np.mean(x_train), np.std(x_train))
    #print(np.mean(x_test), np.std(x_test))

    #Reshape dataset
    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
    x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)
    input_shape = (x_train.shape[1], x_train.shape[2], 1)
    #print(input_shape)
"""
elif encoder_type == "ISI":

    # ISI Encoding.

    # In[20]:


    #Encoder the dataset
    x_train_temp = np.zeros((50000, 32, 32, isiencoderspikecount))

    #print([nonint for nonint in x_train if not isinstance(nonint, int)])
    x_train = np.trunc(x_train).astype(int)
    x_test = np.trunc(x_test).astype(int)
    #print([nonint for nonint in x_train if not isinstance(nonint, int)])
    #print(x_train.shape)
    x_train_temp = ISI_SpikesList[x_train]

    #print(x_train_temp.shape)

    x_test_temp = np.zeros((10000, 32, 32, isiencoderspikecount))

    x_test_temp = ISI_SpikesList[x_test]

    #print(x_test_temp.shape)
    x_train = x_train_temp.reshape(50000, 32, 32, isiencoderspikecount)
    x_test = x_test_temp.reshape(10000, 32, 32, isiencoderspikecount)

    #Normalize the dataset
    x_train = feature_normalize(x_train)
    x_test = feature_normalize(x_test)
    #print(np.mean(x_train), np.std(x_train))
    #print(np.mean(x_test), np.std(x_test))

    input_shape = (x_train.shape[1], x_train.shape[2], isiencoderspikecount)
    #print(input_shape)

"""elif encoder_type == "TTFS-Phase":

    # TTFS-Phase Encoding

    # In[ ]:


    #Encoder the dataset
    x_train_temp = np.zeros((50000, 32, 32))
    x_train = np.trunc(x_train).astype(int)
    x_test = np.trunc(x_test).astype(int)

    #print(x_train.shape)
    #print(x_train)
    #print(x_train[0][1][5])

    for i in range(50000):
      for j in range(32):
        for k in range(32):
          x_train_temp[i][j][k] = TTFS_Phase_SpikesList[x_train[i][j][k]]
    #print(x_train_temp.shape)

    x_test_temp = np.zeros((10000, 32, 32))
    for i in range(10000):
      for j in range(32):
        for k in range(32):
          x_test_temp[i][j][k] = TTFS_Phase_SpikesList[x_test[i][j][k]]
    #print(x_test_temp.shape)
    x_train = x_train_temp.reshape(50000, 32, 32, 1)
    x_test = x_test_temp.reshape(10000, 32, 32, 1)

    #Normalize the dataset
    x_train = feature_normalize(x_train)
    x_test = feature_normalize(x_test)
    #print(np.mean(x_train), np.std(x_train))
    #print(np.mean(x_test), np.std(x_test))

    input_shape = (x_train.shape[1], x_train.shape[2], 1)
    #print(input_shape)
"""
elif encoder_type == "ISI-Phase":

    # ISI-Phase Encoding

    # In[ ]:


    #Encoder the dataset
    x_train_temp = np.zeros((50000, 32, 32, isiencoderspikecount))

    x_train = np.trunc(x_train).astype(int)
    x_test = np.trunc(x_test).astype(int)

    x_train_temp = ISI_Phase_SpikesList[x_train]
    #print(x_train_temp.shape)

    x_test_temp = np.zeros((10000, 32, 32, isiencoderspikecount))

    x_test_temp = ISI_Phase_SpikesList[x_test]
    #print(x_test_temp.shape)
    x_train = x_train_temp.reshape(50000, 32, 32, isiencoderspikecount)
    x_test = x_test_temp.reshape(10000, 32, 32, isiencoderspikecount)

    #Normalize the dataset
    x_train = feature_normalize(x_train)
    x_test = feature_normalize(x_test)
    #print(np.mean(x_train), np.std(x_train))
    #print(np.mean(x_test), np.std(x_test))

    input_shape = (x_train.shape[1], x_train.shape[2], isiencoderspikecount)
    #print(input_shape)


# #Setting model


#Run all blocks including this one and the ones below

model = Sequential([
    layers.RandomFlip("horizontal"),
    # Random zoom (zoom in / out)
])
"""/12/2025: the following is for augmentation (optional)
make_augmentation_pipeline()"""

model.add(Conv2D(16, (3, 3), activation='relu', strides=(1, 1),
    padding='same', input_shape=input_shape)) #16 learned kernels, 3 by 3 in size
model.add(Conv2D(32, (3, 3), activation='relu', strides=(1, 1),
    padding='same')) #32 newly learned kernels, 3 by 3 in size, applied to the 16 feature maps from the last layer; as an output, there are 32 new feature maps (learned combinations of previous 16 feature map chararacteristics)
model.add(Conv2D(64, (3, 3), activation='relu', strides=(1, 1),
    padding='same'))
model.add(MaxPool2D((2, 2))) #Downsizing by taking the maximum value of many 2 by 2 areas
model.add(Conv2D(16, (3, 3), activation='relu', strides=(1, 1),
    padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', strides=(1, 1),
    padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', strides=(1, 1),
    padding='same'))
model.add(MaxPool2D((2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_regularizer = regularizers.l2(0.001) )) #, kernel_regularizer = regularizers.l2(0.001)"""
model.add(Dropout(0.5)) #originally 0.5 # drop 128 neuron outputs (half) from the previous layer
model.add(Dense(128, activation='relu', kernel_regularizer = regularizers.l2(0.001))) #""", kernel_regularizer = regularizers.l2(0.001)"""
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.summary()


# In[23]:

#print("Got to this line")
model.compile(loss='categorical_crossentropy',
     optimizer='adam',
     metrics=['acc'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3) #EarlyStopping prevents overfitting by stopping the test runs after validation accuracy plateaus
T0 = time.time()
history = model.fit(x_train,y_train, epochs=my_epoch, batch_size= 32, validation_data=(x_test, y_test))
#, callbacks=[es]
T1 = time.time()
score = model.evaluate(x_test, y_test, verbose=1)
#print(type(score))
print('Test Score:', score[0])
print('Test Accuracy:', score[1])
print('Training Time:', T1-T0)


# In[ ]:


print(history.history.keys())

print(f"Rate spike ct '{rateencoderspikecount}'")
print(f"ISI spike ct '{isiencoderspikecount}'")
print(f"Epochs '{my_epoch}'")
print(f"Encoder type '{encoder_type}'")

num_xticks = 20 + 1
fig, ax = plt.subplots(nrows = 2, ncols = 2, figsize = (16, 10) )
train_x = range(1, len(history.history["acc"])+1)
train_y = history.history["acc"]
train_loss = history.history["loss"]
ax[0,0].plot(train_x, train_y)
ax[0,0].set_xlabel("Epochs")
ax[0,0].set_ylabel("Training Accuracy")
ax[0,0].set_xticks(range(1, num_xticks))
ax[0,0].tick_params(axis = "both", labelsize = 12)
ax[0,0].set_yticks(np.linspace(start = min(train_y), stop = max(train_y), num = 10))

val_x = range(1, len(history.history["val_acc"])+1)
val_y = history.history["val_acc"]
ax[0,1].plot(val_x , val_y)
ax[0,1].set_xlabel("Epochs")
ax[0,1].set_ylabel("Validation Accuracy")
ax[0,1].set_xticks(range(1, num_xticks))
ax[0,1].set_yticks(np.linspace(start = min(val_y), stop = max(val_y), num = 10))
ax[0,1].tick_params(axis = "both", labelsize = 12)

train_loss = history.history["loss"]
ax[1,0].plot(train_x, train_loss)
ax[1,0].set_xlabel("Epochs")
ax[1,0].set_ylabel("Training Loss")
ax[1,0].set_xticks(range(1, num_xticks))
ax[1,0].tick_params(axis = "both", labelsize = 12)
ax[1,0].set_yticks(np.linspace(start = min(train_loss), stop = max(train_loss), num = 10))

val_loss = history.history["val_loss"]
ax[1,1].plot(train_x, val_loss)
ax[1,1].set_xlabel("Epochs")
ax[1,1].set_ylabel("Validation Loss")
ax[1,1].set_xticks(range(1, num_xticks))
ax[1,1].tick_params(axis = "both", labelsize = 12)
ax[1,1].set_yticks(np.linspace(start = min(val_loss), stop = max(val_loss), num = 10))

#plt.show()


# In[ ]:




